In [1]:
import sys 
sys.path.append('..')
from qiskit import IBMQ

# you might need IBMQ account fill the below format 
## https://quantum-computing.ibm.com/
#IBMQ.save_account('your account')


In [8]:
# math tools
import numpy as np 
from scipy import stats 
import os
import copy
# class
from argparse import Namespace

# neural net 
import torch 
import torch.nn as nn
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

from neural.net import VonNeumannEP, train, validate

# iteration
from tqdm.notebook import tqdm


## xxzchain 
from xxzchain.xxzchain import groundXXZ, entangleandvislist, xxzansatz

# quantum circuit 
from qiskit import Aer
backend = Aer.get_backend("aer_simulator") # set simulator
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.algorithms.optimizers import COBYLA, ADAM, SLSQP, SPSA, GradientDescent
from qiskit.quantum_info import random_statevector, random_density_matrix, Statevector
from qiskit.circuit.library import EfficientSU2
## custom lib 
from cirquit.tools import data_processing, initial_entangle_EP
from cirquit.CircuitModelMk4 import HybridCircuit

#from qiskit import tools
from qiskit import quantum_info
import qutip as qt

# graph tool 
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

# date for mkdir 
from datetime import date

import time

In [9]:
import pickle
def save_obj(obj, name ):
    """
    inputs
    obj 
    name string
    """
    with open( name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open( name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [10]:
import numpy as np
# quantum circuit 
from qiskit import Aer
import copy
backend = Aer.get_backend("aer_simulator") # set simulator
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from cirquit.tools import data_processing, initial_entangle_EP
from qiskit.algorithms.optimizers import COBYLA, ADAM, SLSQP, SPSA
from qiskit.quantum_info import random_statevector, random_density_matrix
from qiskit.circuit.library import EfficientSU2
from torch.utils.data import DataLoader
from neural.net import VonNeumannEP, train, validate
import torch 
#from qiskit import tools
from qiskit import quantum_info

from qiskit import IBMQ


class VQSE:
    def __init__(self, opt):
        self.n_qubit = opt.n_qubit # number of qubits that we observe
        self.n_clbit = opt.n_clbit # number of classical bits that we observe
        self.ini_state = opt.ini_state # initial state
        self.reps = opt.reps # circuit depth
        self.n_shots = opt.n_shots # shots for one ensemble
        self.num_ini_qubit = opt.num_ini_qubit # number of initial qubits
        self.num_ini_clbit = opt.num_ini_clbit # number of inital classical bits
        self.n_token = opt.n_token
        
        # added 6th June 
        # for VQSE cost ftn
        self.r1 = opt.r1
        self.VQSEdelta = opt.VQSEdelta
        self.GLratio = opt.GLratio      
        self.optim = opt.optim 
        try:
            self.m_indexlist = opt.m_indexlist
        except:
            print('no m_lindexlist')
        try:
            self.m_keylist = opt.m_keylist
        except:
            print('no m_keylist')
        
    def build_qc(self, parameters):
        q = QuantumRegister(self.num_ini_qubit, name="q")
        c = ClassicalRegister(self.num_ini_clbit, name="c")
        qc = QuantumCircuit(q, c)
        qc.initialize(self.ini_state)
        shift = 0 
        #####################################
        for i in np.arange(self.n_qubit):
            k = i
            qc.ry(parameters[k],q[i])
        aux_par_num = k+1 
        for i in np.arange(int(self.n_qubit/2)):
            qc.cz(q[2*i+shift],q[(2*i+1+shift)%self.n_qubit])
        for i in np.arange(self.n_qubit):
            k =i + aux_par_num
            qc.ry(parameters[k],q[i])
        aux_par_num = k+1    
        shift+=1
        shift%=2
        for j in np.arange(self.reps-1):
        #####################################
            for i in np.arange(self.n_qubit):
                k =i + aux_par_num
                qc.ry(parameters[k],q[i])
            aux_par_num = k+1  
            for i in np.arange(int(self.n_qubit/2)):
                qc.cz(q[2*i+shift],q[(2*i+1+shift)%self.n_qubit])
            #####################################
            for i in np.arange(self.n_qubit):
                k =i + aux_par_num
                qc.ry(parameters[k],q[i])
            aux_par_num = k+1    
            shift+=1
            shift%=2
        qc.save_statevector() 
        qc.measure(q[:self.n_qubit], c[:self.n_qubit])
        return qc 
    def exactHamcostfunction(self,parameters):
        qc = self.build_qc(parameters)
        result = backend.run(qc, shots = 1).result()
        _counts = result.get_counts()
        _data, _data_table, entropy = data_processing(_counts, self.n_qubit)
        _fin_statevec  = result.get_statevector(qc)
        _fin_reduced_state = quantum_info.partial_trace(_fin_statevec,
                                                        list(np.arange(self.n_qubit,self.num_ini_qubit) )# remove quits n_qubits+1 ~ ini_qubits
                                                       )

        
        Hl, _r_list = self.gen_Hl(_fin_reduced_state.num_qubits, 
                                  _fin_reduced_state)
        #print(Hl, _r_list)
        
        
        m = _fin_reduced_state.num_qubits + 1 ## m = n+1 ! look the paper VQSE 
        q_list = self.gen_qlist(m,_r_list) # large population * large q! (small E )
        #print('q_list',q_list)
        

        Hg = 1
        for idx, k in enumerate(self.m_indexlist):
            diag_comp = _fin_reduced_state.data[k,k].real
            cof = q_list[idx]
            Hg -=  cof * diag_comp


        cost_ftn = Hg * self.GLratio + Hl * (1 - self.GLratio)        
        return cost_ftn 

    def exactRenyicostfunction(self,parameters):
        _alpha = self.alpha # added term for Renyi cost ftn 
        qc = self.build_qc(parameters)
        result = backend.run(qc, shots = 1).result()
        _counts = result.get_counts()
        _data, _data_table, entropy = data_processing(_counts, self.n_qubit)
        _fin_statevec  = result.get_statevector(qc)
        _fin_reduced_state = quantum_info.partial_trace(_fin_statevec,
                                                       list(self.n_qubit + np.arange(self.n_qubit)))
        test_sum = 0
        cost_ftn = 0
        for i in np.arange(2**_fin_reduced_state.num_qubits):
            for j in np.arange(2**_fin_reduced_state.num_qubits):
                if i==j:
                    pii = _fin_reduced_state.data[i,i]
                    pii = pii.real
                    cost_ftn += pii ** _alpha
        cost_ftn /= _alpha *(1-_alpha)
        return cost_ftn
    def exactcostfunction(self,parameters):
        qc = self.build_qc(parameters)
        result = backend.run(qc, shots = 1).result()
        _counts = result.get_counts()
        _data, _data_table, entropy = data_processing(_counts, self.n_qubit)
        _fin_statevec  = result.get_statevector(qc)
        _fin_reduced_state = quantum_info.partial_trace(_fin_statevec,
                                                        list(np.arange(self.n_qubit,self.num_ini_qubit) )# remove quits n_qubits+1 ~ ini_qubits
                                                       )
        test_sum = 0
        cost_ftn = 0
        for i in np.arange(2**_fin_reduced_state.num_qubits):
            for j in np.arange(2**_fin_reduced_state.num_qubits):
                if i == j:
                    pii = _fin_reduced_state.data[i,i]
                    pii = pii.real
                    test_sum += pii
                    cost_ftn += - np.log( pii ) * pii
        cost_ftn += np.log(test_sum)
        return cost_ftn
    def gen_qlist(self,m, r_list):
        """
        create qlist for Hg from r_list of Hl 
        """
        LocalHam =  qt.tensor([  qt.qeye(2) for j in np.arange(self.n_qubit )])
        for i in np.arange(self.n_qubit ):
            LocalHam += -  r_list[i] * qt.tensor([ (1/2) * qt.sigmaz() if j == i else qt.qeye(2) 
                                                for j in np.arange(self.n_qubit )])
        result = 1 - LocalHam.eigenenergies()[:m]
        return result 
    def gen_Hl(self,num_view_qubits, partial_state):
        """
        returns
        result = Hl value
        r_list = r value list, for cal of Hl and Hg
        """
        
        Hl = 1.
        traceoverlist = np.arange(num_view_qubits) 
        #print(traceoverlist)
        _updownlist = []
        for k in np.arange(num_view_qubits):
            l = list(traceoverlist)
            l.remove(k)
            #print(l)
            _temp_state = quantum_info.partial_trace(partial_state,
                                                     l# remove quits n_qubits+1 ~ ini_qubits
                                                     )
            assert _temp_state.num_qubits == 1 
            _updownlist.append( (_temp_state.data[0,0].real, _temp_state.data[1,1].real) )
        #print(_updownlist)
        r_list = [] # build r list for coef of Sz 
        for k in  np.arange(self.n_qubit):
            val = self.r1 + self.VQSEdelta * (k)
            r_list.append(val)
        
        for idx, rho in enumerate(_updownlist): # Sz evaluation
            Hl+= - r_list[idx] * ( rho[0] - rho[1] )/2 # i added half factor 
        #print(idx, 'idx!!', rho[0], 'population!!' )
        return Hl, r_list 
    def gen_idxlist(self, reduced_state):
        _, _r_list = self.gen_Hl(reduced_state.num_qubits, 
                                  reduced_state)
        
        m = reduced_state.num_qubits + 1 ## m = n+1 ! look the paper VQSE 
        
        evallist = []
        for i in np.arange(2**reduced_state.num_qubits):
            evallist.append(reduced_state.data[i,i].real)
        evallist = np.array(evallist)
        mevallist = evallist.copy()
        mevallist.sort() 
        mevallist = mevallist[::-1][:m] # m diaognal element from lagest one! 
        mindex_list = []
        for i in mevallist:
            mindex_list.append(np.where(evallist ==i )[0].squeeze() )
        return mindex_list, mevallist
    def finite_gen_Hl(self, counts):
        """
        1- \sum r_i * <Sz_i >
        """
        r_list = [] # build r list for coef of Sz 
        for k in  np.arange(self.n_qubit):
            val = self.r1 + self.VQSEdelta * (k)
            r_list.append(val)

        #print(r_list)

        temp_keys = []
        energy_list = [] # <S_z > list ! 
        for key in list(counts.keys()):
            key = key[-self.n_qubit:]
            temp_keys.append( key )
            term_for_second = 0
            for idx, state in enumerate(key):
                #print(state)
                state = - int(state)
                state += 1/2.
                term_for_second += state * r_list[idx]
            energy_list.append(term_for_second)
        #print(temp_keys)
        #print(energy_list)
        proplist = list(counts.values()) 
        proplist /= np.sum(proplist)
        #print(proplist)
        Hl = 1.
        Hl -= np.inner( proplist , energy_list)
        return Hl, r_list 
    def finite_gen_keylist(self, counts):
        test = np.array(list(counts.values()))
        test.sort()
        test = test[::-1]
        #print(test[:opt.n_qubit+1]) # sort? 
        mkeys = []
        for _, cont in enumerate(test[:self.n_qubit+1]):
            idx = np.where( np.array(list(counts.values())) == cont)[0][0]
            mkeys.append(list(counts.keys())[idx])

        return mkeys
    
    def finiteHamcostfunction(self,parameters):
        qc = self.build_qc(parameters)
        result = backend.run(qc, shots = self.n_shots).result()
        _counts = result.get_counts()
        
        Hl, _r_list = self.finite_gen_Hl(_counts)
       
        m = self.n_qubit + 1 ## m = n+1 ! look the paper VQSE 
        q_list = self.gen_qlist(m, _r_list) # large population * large q! (small E )
        #print('q_list',q_list)
        

        Hg = 1.
        for idx, k in enumerate(self.m_keylist):
            try:
                diag_comp =_counts[k]/self.n_shots
            except:
                diag_comp = 0.
            cof = q_list[idx]
            #print(diag_comp, cof)
            Hg -=  cof * diag_comp


        cost_ftn = Hg * self.GLratio + Hl * (1 - self.GLratio)        
        return cost_ftn 


## make initial state 

In [18]:
opt = Namespace()

## for qc 
opt.n_shots = 30000 #000#00 #
opt.num_ini_qubit = 8
opt.num_ini_clbit = opt.num_ini_qubit
opt.n_qubit = 3 # 4
opt.n_clbit = opt.n_qubit #
opt.reps = 8 
opt.num_parameters = int( 2 * opt.n_qubit *(opt.reps+1) ) #
opt.qstate_seed = 10 #


opt.state_dim = 2 ** opt.num_ini_qubit #

# xxzchain 
#opt.num_mipt_layer = 2 * opt.num_ini_qubit
opt.delta  = 0.05  # probability for MIPT
opt.mag = 0.1

print('state dim is ',opt.state_dim)
print('depth is ', opt.reps)
print('# of pars is ', opt.num_parameters)

# VQSE 
opt.r1 = 0.2 # r>delta 
opt.VQSEdelta = 0.01

# neural net opt  
opt.n_token = int( 2 ** opt.n_qubit ) #

# the nubmer of events 01, 11, 10, 00 for our reduced state
opt.n_hidden = 256 #
opt.n_layer = 3 #
opt.batch_size = opt.n_shots #0 #4096 #
#opt.evaluation_batch_size = 2**10 # 1024\
opt.device = 'cuda:0' # not used for VQSE 
opt.lr = 1e-5 # # not used for VQSE 
opt.wd = 5e-5 # # not used for VQSE 
opt.record_freq = 100 # # not used for VQSE 
opt.seed = 398 ## not used for VQSE 
opt.datasize = opt.n_shots
opt.model = VonNeumannEP(opt).to(opt.device) # not used for VQSE 

# train strategy paramters 
opt.maxiter = 25 #25  
opt.circuit_repeats = 10 #30 for 4 qubits 
opt.n_train_repeats = 5 # not used for VQSE 
opt.ini_nn_iter = 4000 #10000 # not used for VQSE 
opt.inter_nn_iter = 100 # #10000 # not used for VQSE 


# Adam optimizer 
opt.optim = torch.optim.Adam(opt.model.parameters(), opt.lr, weight_decay=opt.wd)

# save
opt.directory = '../notebooks/data/VQSE'
print(opt.directory)
if not os.path.exists(opt.directory):
    os.makedirs(opt.directory)

state dim is  256
depth is  8
# of pars is  54
../notebooks/data/VQSE


In [19]:
def finite_VQSE_trial(opt,seed, lr ):
    """
    input opt, seed, lr 
        seed for initial circuit parameters
        lr for circuit tuning 
    returns minE, exact_learning_curve, minlist, EP
              
    """
    np.random.seed(seed)
    ini_parameters = 2. * np.pi * np.random.uniform(size = opt.num_parameters)
    qc_parameters = ini_parameters
    save_parameters  = qc_parameters
    optimizer = GradientDescent(learning_rate = lr , maxiter = opt.maxiter) #ADAM(maxiter = 10)
    t2 = time.time()

    opt.GLratio = 0.
    VQSEClass = VQSE(opt)
    qc = VQSEClass.build_qc(ini_parameters)#.exactHamcostfunction(ini_parameters)

    result = backend.run(qc, shots = opt.n_shots).result()
    temp_state = result.get_statevector(qc)
    reduced_state = quantum_info.partial_trace(temp_state,
                                                list(np.arange(opt.n_qubit,opt.num_ini_qubit) )# remove quits n_qubits+1 ~ ini_qubits
                                               )

    eigenvalue_list = np.linalg.eigh(reduced_state.data)[0][::-1]
    print(eigenvalue_list[:opt.n_qubit+1])

    VQSEClass.m_keylist = VQSE(opt).finite_gen_keylist(result.get_counts())
    #VQSEClass = VQSE(opt)
    print('mkeylist', VQSEClass.m_keylist)

    minE = 100. 
    minlist = 0.
    exact_learning_curve = []
    exact_learning_curve.append(VQSEClass.finiteHamcostfunction(qc_parameters))
    for QC_iter in np.arange(opt.circuit_repeats ):
        opt.GLratio = (QC_iter+ 1 )/opt.circuit_repeats
        VQSEClass = VQSE(opt)
        qc = VQSEClass.build_qc(qc_parameters)#.exactHamcostfunction(ini_parameters)

        result = backend.run(qc, shots = opt.n_shots).result()
        temp_state = result.get_statevector(qc)
        reduced_state = quantum_info.partial_trace(temp_state,
                                                    list(np.arange(opt.n_qubit,opt.num_ini_qubit) )# remove quits n_qubits+1 ~ ini_qubits
                                                   )

        # update m_list ! 
        counts = result.get_counts() # counts with updated qc parameters! 
        VQSEClass.m_keylist = VQSE(opt).finite_gen_keylist(counts) # update mkey! 
        print('mkeylist', VQSEClass.m_keylist)
        result = optimizer.minimize(fun = VQSEClass.finiteHamcostfunction, # minimize! 
                                    x0  = qc_parameters)
        qc_parameters = result.x
        opt.exactparameters = qc_parameters
        energy = VQSEClass.finiteHamcostfunction(qc_parameters)
        EP = VQSEClass.exactcostfunction(qc_parameters)
        plist = []
        for i in counts.values():
            plist.append(i/opt.n_shots)
        finEP = 0
        for p in plist:
            finEP += - p * np.log(p)
        exact_learning_curve.append(energy)
        print('exact Energy' ,energy,
               'exact EP', EP, 
               ' EP from shots ', finEP,
              'time: ', time.time() - t2 )
        plist=np.sort(plist)
        plist=plist[::-1]
        print('large prop list ', plist)
    return energy, exact_learning_curve, plist, EP, finEP

In [20]:
def exacteigenvalues(opt):
    """
    input opt, seed, lr 
        seed for initial circuit parameters
        lr for circuit tuning 
    returns minE, exact_learning_curve, minlist, EP
              
    """
    #np.random.seed(seed)
    ini_parameters = ini_parameters = 2. * np.pi * np.random.uniform(size = opt.num_parameters)
    qc_parameters = ini_parameters
    save_parameters  = qc_parameters
    t2 = time.time()

    opt.GLratio = 0.
    VQSEClass = VQSE(opt)
    qc = VQSEClass.build_qc(ini_parameters)#.exactHamcostfunction(ini_parameters)

    result = backend.run(qc, shots = opt.n_shots).result()
    temp_state = result.get_statevector(qc)
    reduced_state = quantum_info.partial_trace(temp_state,
                                                list(np.arange(opt.n_qubit,opt.num_ini_qubit) )# remove quits n_qubits+1 ~ ini_qubits
                                               )

    eigenvalue_list = np.linalg.eigh(reduced_state.data)[0][::-1]
    return eigenvalue_list

## magnetic field swip 

In [21]:
opt.GLratio = 0.
opt.num_try = 5 
opt.circuitval = 0.05 

for idx, opt.mag in enumerate([0.0]):
#for idx, opt.mag in enumerate([0.0,0.5,1.0,1.5,2.0,2.5,3.0]):  # for reproducing data used in QNEE reserach  
    # initial setup for state initialization 
    opt.ini_state = groundXXZ(opt.num_ini_qubit, opt.delta, opt.mag).full().flatten()

    reduced_state = quantum_info.partial_trace(opt.ini_state,
                                               list(np.arange(opt.n_qubit,opt.num_ini_qubit) )# remove quits n_qubits+1 ~ ini_qubits
                                               )
    
    opt.eigenvalue_list = np.linalg.eigh(reduced_state.data)[0][::-1]
    opt.EP2 = quantum_info.entropy(reduced_state, base = np.e) 
    print('EP is', opt.EP2)
        
    VQSE_trial_list = []
    for i in np.arange(opt.num_try):
        a,b,c,d, e = finite_VQSE_trial(opt,i, opt.circuitval )
        VQSE_trial_list.append( (a,b,c,d, e) )
        print('seed '+str(i)+'ends')
        
    opt.VQSE_trial_list = VQSE_trial_list

    print(opt.directory)
    if not os.path.exists(opt.directory):
        os.makedirs(opt.directory)
    if True: # set True for saving data
        save_obj(opt, opt.directory + '/Finite_VQSE_{}qubits_XXZdelta{}mag{}trial{}qlr{}'.format(
                         opt.n_qubit, opt.delta, opt.mag, opt.num_try, opt.circuitval ) )     

EP is 1.0166576561462088
no m_lindexlist
no m_keylist
[0.46275664 0.46275664 0.01915113 0.01915113]
no m_lindexlist
no m_keylist
mkeylist ['00000101', '00000111', '00000110', '00000001']
no m_lindexlist
no m_keylist
no m_lindexlist
no m_keylist
mkeylist ['00000101', '00000111', '00000110', '00000001']
exact Energy 0.95999005 exact EP 1.977369592986866  EP from shots  1.9844067336875635 time:  60.13616394996643
large prop list  [0.2345     0.2003     0.12683333 0.10673333 0.1015     0.08913333
 0.07323333 0.06776667]
no m_lindexlist
no m_keylist
no m_lindexlist
no m_keylist
mkeylist ['00000101', '00000001', '00000010', '00000100']
exact Energy 0.8988493666666668 exact EP 1.7915471975223582  EP from shots  1.9760568417854736 time:  119.76351714134216
large prop list  [0.21403333 0.19026667 0.1432     0.1333     0.1306     0.07376667
 0.0701     0.04473333]
no m_lindexlist
no m_keylist
no m_lindexlist
no m_keylist
mkeylist ['00000001', '00000000', '00000010', '00000101']
exact Energy 0.85